<a href="https://colab.research.google.com/github/aaronpetryio/random/blob/main/mmlm_model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Import Libraries

In [1]:
import pandas as pd
import os
import re

#### Connect to Kaggle and download data

In [ ]:
!pip install -q kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"aaronpetryio","key":"ac3f073bb1dfa9030d82b13d6f357d21"}'}

In [3]:
!mkdir ~/.kaggle

In [4]:
!cp kaggle.json ~/.kaggle/

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c warmup-round-march-machine-learning-mania-2023

 44% 13.0M/29.2M [00:00<00:00, 130MB/s]
100% 29.2M/29.2M [00:00<00:00, 187MB/s]


In [7]:
!unzip warmup-round-march-machine-learning-mania-2023.zip

Archive:  warmup-round-march-machine-learning-mania-2023.zip
  inflating: Cities.csv              
  inflating: Conferences.csv         
  inflating: MConferenceTourneyGames.csv  
  inflating: MGameCities.csv         
  inflating: MMasseyOrdinals.csv     
  inflating: MNCAATourneyCompactResults.csv  
  inflating: MNCAATourneyDetailedResults.csv  
  inflating: MNCAATourneySeedRoundSlots.csv  
  inflating: MNCAATourneySeeds.csv   
  inflating: MNCAATourneySlots.csv   
  inflating: MRegularSeasonCompactResults.csv  
  inflating: MRegularSeasonDetailedResults.csv  
  inflating: MSeasons.csv            
  inflating: MSecondaryTourneyCompactResults.csv  
  inflating: MSecondaryTourneyTeams.csv  
  inflating: MTeamCoaches.csv        
  inflating: MTeamConferences.csv    
  inflating: MTeamSpellings.csv      
  inflating: MTeams.csv              
  inflating: SampleSubmissionWarmup.csv  
  inflating: WGameCities.csv         
  inflating: WNCAATourneyCompactResults.csv  
  inflating: WNCAATourn

#### EDA / Data Cleaning

In [13]:
df_results_w = pd.read_csv("WRegularSeasonCompactResults.csv")
df_results_m = pd.read_csv("MRegularSeasonCompactResults.csv")

In [14]:
df_results_full = pd.concat([df_results_w, df_results_m], axis=0)

In [16]:
df_results_full.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,18,3104,91,3202,41,H,0
1,1998,18,3163,87,3221,76,H,0
2,1998,18,3222,66,3261,59,H,0
3,1998,18,3307,69,3365,62,H,0
4,1998,18,3349,115,3411,35,H,0


#### Add Data Columns

In [20]:
df_results_full['ScoreGap'] = df_results_full['WScore'] - df_results_full['LScore']

In [40]:
num_wins = df_results_full.groupby(['Season', 'WTeamID']).count()
num_wins = num_wins.reset_index()[['Season', 'WTeamID', 'DayNum']].rename(columns = {"WTeamID": "TeamID", "DayNum": "NumWins"})

In [42]:
num_wins.head()

,Season,TeamID,NumWins
0,1985,1102,5
1,1985,1103,9
2,1985,1104,21
3,1985,1106,10
4,1985,1108,19


In [51]:
num_losses = df_results_full.groupby(['Season', 'LTeamID']).count().reset_index()
num_losses = num_losses[['Season', 'LTeamID', 'DayNum']].rename(columns = {"LTeamID": "TeamID", "DayNum": "NumLosses"})

In [52]:
num_losses.head()

,Season,TeamID,NumLosses
0,1985,1102,19
1,1985,1103,14
2,1985,1104,9
3,1985,1106,14
4,1985,1108,6


In [53]:
win_gap = df_results_full.groupby(['Season', 'WTeamID']).mean().reset_index()
win_gap = win_gap[['Season', 'WTeamID', 'ScoreGap']].rename(columns = {"WTeamID": "TeamID", "ScoreGap": "WinGap"})

In [56]:
loss_gap = df_results_full.groupby(['Season', 'LTeamID']).mean().reset_index()
loss_gap = loss_gap[['Season', 'LTeamID', 'ScoreGap']].rename(columns = {"LTeamID": "TeamID", "ScoreGap": "LossGap"})

#### Create a data

In [58]:
df_results_w = df_results_full.groupby(['Season', 'WTeamID']).count().reset_index()[['Season', 'WTeamID']].rename(columns={"WTeamID": "TeamID"})
df_results_l = df_results_full.groupby(['Season', 'LTeamID']).count().reset_index()[['Season', 'LTeamID']].rename(columns={"LTeamID": "TeamID"})

In [60]:
df_features_full = pd.concat([df_results_w, df_results_l], 0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)

<ipython-input-60-60ecc23d295d>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  df_features_full = pd.concat([df_results_w, df_results_l], 0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)


In [62]:
df_features_full.describe()

,Season,TeamID
count,21428.000000,21428.000000
mean,2007.286681,2103.063282
std,10.259229,988.882093
min,1985.000000,1101.000000
25%,2000.000000,1257.000000
50%,2008.000000,1410.000000
75%,2016.000000,3245.000000
max,2023.000000,3477.000000
